In [168]:

import numpy as np
import cv2
import os
import sys
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision.transforms as transforms
import torchvision.datasets as datasets



In [169]:

#Takes images in concat_data and flatten them into a 1D array
def flatten_images(category):
    #Get the list of images in the directory
    images = os.listdir("../concat_data/%s" % category)
    #Create an empty list to store the flattened images
    flattened_images = []
    #Iterate through the images
    for image in images:
        #check extension of the image
        if image.split(".")[-1] == "jpg":
            #Read the image
            img = cv2.imread("../concat_data/%s/%s" %(category, image), 0)
            #Flatten the image
            # img = img.flatten()
            #Add the flattened image to the list
            flattened_images.append(img)

    #Return the list of flattened images
    return flattened_images

In [170]:
# add images of each category into an array


images_dict = {'focused': flatten_images('focused'),
               'happy': flatten_images('happy'),
               'neutral': flatten_images('neutral'),
               'surprised': flatten_images('surprised')}

# tokensize labels
labels = {'focused': 0,
          'happy': 1,
          'neutral': 2,
          'surprised': 3}


# concatencate all the data with respective labels
x = []
y = []
for key in images_dict:
    for image in images_dict[key]:
        x.append(image)
        y.append(labels[key])
        
# split into training, validation, and testing
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size =0.30, random_state=42)

# split x_temp and y_temp into validation and testing\
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size =0.50, random_state=42)




In [171]:
# Convert all the data into tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train)

x_valid_tensor = torch.tensor(x_valid, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test)



In [172]:
# Hyperparameters and settings
batch_size = 64
test_batch_size = 64
input_size = 1 # because there is only one channel 
output_size = 4
num_epochs = 10
learning_rate = 0.001



In [173]:
# x_numpy = np.array(x)/255.0 # Normalized values 
# y_numpy = np.array(y)
# x_reshaped = x_numpy.reshape(len(x_numpy), 1, 48, 48)
# # print(x_reshaped)
# # Convert numpy arrays to PyTorch tensors
# images_tensor = torch.tensor(x_numpy, dtype=torch.float32)
# images_tensor = images_tensor.unsqueeze(1)
# labels_tensor = torch.tensor(y_numpy, dtype=torch.long)
images_tensor = torch.tensor(x_train, dtype=torch.float32)
images_tensor = images_tensor.unsqueeze(1)
labels_tensor = torch.tensor(y_train, dtype=torch.long)

images_test_tensor = torch.tensor(x_valid, dtype=torch.float32)
images_test_tensor = images_test_tensor.unsqueeze(1)
labels_test_tensor = torch.tensor(y_valid, dtype=torch.long)

print(images_tensor[3][0])
# Create a TensorDataset
dataset = td.TensorDataset(images_tensor, labels_tensor)

# Create DataLoader for batching and shuffling
batch_size = 32
data_loader = td.DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_loader = td.DataLoader(dataset, batch_size=batch_size, shuffle=True)

tensor([[ 27.,  15.,  18.,  ...,  28.,  53.,  33.],
        [ 31.,  31.,  29.,  ...,  23.,  22.,  23.],
        [ 41.,  41.,  40.,  ...,  29.,  26.,  31.],
        ...,
        [254., 253., 252.,  ..., 149., 148., 153.],
        [251., 255., 253.,  ..., 154., 156., 149.],
        [254., 251., 239.,  ..., 164., 152., 141.]])


In [174]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(12 * 12 * 64, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        # print(x.shape)
        # flatten
        x = x.view(x.size(0), -1)
        # print(x.shape)
        # fc layer
        x = self.fc_layer(x)
        return x


In [175]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [176]:
print(len(x_train_tensor))

1400


In [177]:
total_step = len(data_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Train accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        # print(i)
        if (i + 1) % 62 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
            .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
            (correct / total) * 100))

In [178]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Test Accuracy of the model on the 10000 test images: {} %'
        .format((correct / total) * 100))

Test Accuracy of the model on the 10000 test images: 100.0 %
Test Accuracy of the model on the 10000 test images: 98.4375 %
Test Accuracy of the model on the 10000 test images: 98.95833333333334 %
Test Accuracy of the model on the 10000 test images: 99.21875 %
Test Accuracy of the model on the 10000 test images: 99.375 %
Test Accuracy of the model on the 10000 test images: 98.95833333333334 %
Test Accuracy of the model on the 10000 test images: 99.10714285714286 %
Test Accuracy of the model on the 10000 test images: 99.21875 %
Test Accuracy of the model on the 10000 test images: 99.30555555555556 %
Test Accuracy of the model on the 10000 test images: 99.0625 %
Test Accuracy of the model on the 10000 test images: 99.14772727272727 %
Test Accuracy of the model on the 10000 test images: 98.95833333333334 %
Test Accuracy of the model on the 10000 test images: 98.79807692307693 %
Test Accuracy of the model on the 10000 test images: 98.66071428571429 %
Test Accuracy of the model on the 10000